sources for future projects
bidspr
cor3
aaa website
aaa infrastructure plan
newspaper articles

In [273]:
import pandas as pd
import numpy as np
from PyPDF2 import PdfReader
import re
from pathlib import Path
import tabula
from collections import defaultdict

In [507]:
asset_types = {
    'B' : 'Buildings',
    'D' : 'Damns',
    'D&T-WL' : 'Distribution & Transmission Water Lines',
    'OO' : 'Ocean Outflow',
    'PPTD' : 'Project TBD',
    'R' : 'Reservoir',
    'RWW' : 'Raw Water Well',
    'T' : 'Telemetry',
    'TSL' : 'Trunk Sewer Lines',
    'WM' : 'Water Meters',
    'WPS' : 'Water Pump Stations',
    'WST' : 'Water Storage Tanks',
    'WST & WPS' : 'Water Storage Tanks and Water Pump Stations',
    'WTP' : 'Water Treatment Plant',
    'WWPS' : 'Wastewater Pump Station',
    'WWTP' : 'Wastewater Treatment Plant'
}

asset_types_eng_abbr = [k for k in asset_types.keys()]
asset_types_eng_full = [v[0] for v in asset_types.values()]

In [500]:
#create df from tables in pdf
df_notable_projects = pd.DataFrame()

#create dict of tables with info about the tables
tables = {
    'Near-Term' : {'page_nums': '22-29', 'yrs': '2021-2023'},
    'Mid-Term' : {'page_nums': '31-33', 'yrs': '2024-2027'},
    'Long-Term' : {'page_nums': '35-36', 'yrs': '2028 & beyond'},
}

#iterate through dict of tables in pdf
for table_name, inner_dicts in tables.items():

    #initialize df_table
    df_table = pd.DataFrame()
    
    #extract current table from pdf
    dfs = tabula.read_pdf("aaa_faast_workplan.pdf", pages=inner_dicts['page_nums'])

    #iterate through pages from table
    for page_num, df_page in enumerate(dfs):

        df_page = df_page.drop([0,1])
        
        #add page num col
        df_page['page'] = page_num + 1
                
        #rename table columns
        df_page.columns = [col.lower().replace(' ', '_').replace('project', 'cip').replace('asset','asset_type').replace('sow', 'sow_submittal').replace('total_cost', 'total_cost_estimate') for col in df_page.columns]
        
        #concat table from page into whole table dataframe
        df_table = pd.concat([df_table, df_page], ignore_index=True)
        
        #add col with table name
        df_table['timeframe'] = table_name
    
    df_notable_projects = pd.concat([df_notable_projects, df_table], ignore_index=True)

In [501]:
#check for wraparound text in each column to determin which columns I need to perform combine_rows

#create df with only columns / rows where project_# is null and column/row is not null 
df_wraparound_rows = df_notable_projects[df_notable_projects['cip_#'].isna()].dropna(axis=1, how='all')

#declare default dict and a list of cols to exclude (because i know there is data in each row)
dd = defaultdict(list)
cols_to_exclude = ['page', 'timeframe'] 

#iterate through cols, and add to dd col : series excluding na 
for col in df_wraparound_rows:
    if not col in cols_to_exclude: 
        dd[col] = df_wraparound_rows[col].dropna()

cols_with_wrap = dd.keys()
cols_with_wrap

dict_keys(['asset_type', 'description', 'total_cost_estimate'])

In [502]:
#front fill cip_#
df_notable_projects['cip_#'] = df_notable_projects['cip_#'].fillna(method='ffill')

#concat text from columns with wraparound text
def concat_text(ser):
    return ' '.join([value for value in ser if str(value) != 'nan'])

for col in cols_with_wrap:
    df_notable_projects[col] = df_notable_projects.groupby('cip_#')[col].transform(concat_text)

#dropna
df_notable_projects = df_notable_projects.dropna()


asset_type                                                     WST & WPS
cip_#                                                        CIP.2076044
description            Installation of MCC at Medio Millon Arecibo WS...
a&e_start                                                        2023-Q3
sow_submittal                                                    2023-Q4
total_cost_estimate                                                $0.40
page                                                                   3
timeframe                                                      Near-Term
Name: 80, dtype: object

In [503]:
#sanity check: see if any data points are NaN 
df_notable_projects[df_notable_projects.isna()].count()

asset_type             0
cip_#                  0
description            0
a&e_start              0
sow_submittal          0
total_cost_estimate    0
page                   0
timeframe              0
dtype: int64

In [504]:
df_notable_projects.sample(10)

,asset_type,cip_#,description,a&e_start,sow_submittal,total_cost_estimate,page,timeframe
178,WTP,CIP.3156094,Replacement of RWI at Mameyes River for the Yu...,2022-Q3,2023-Q4,$15.55,5,Near-Term
240,WTP,CIP.4646004,Raw Water Wells Closure/ Salinas WTP (FEMA-404),2022-Q1,2024-Q4,Proposed HMP,6,Near-Term
94,D&T-WL,CIP.2269000,"Improvements to Water Distribution System, Dor...",2023-Q2,2023-Q4,$1.43,3,Near-Term
124,WTP,CIP.2426100,Rehabilitation of Lares Nueva Espino WTP and RWI,2021-Q3,2022-Q1,$27.03,4,Near-Term
151,WTP,CIP.2739001,Rehabilitation of Lago Viví RWI,2021-Q3,2023-Q2,$5.59,4,Near-Term
444,WWPS,CIP.1009204,Rehabilitation of Wastewater Pump Stations Tan...,2028-Q2,2028-Q3,$6.90,1,Long-Term
113,WTP,CIP.2386049,Elimination of Jayuya Canalizo WTP and RWI,2022-Q4,2024-Q4,$6.67,4,Near-Term
58,TSL,CIP.1705003,"Rehabilitation of Urb. Levittown TS, Toa Baja",2023-Q3,2023-Q4,$0.67,2,Near-Term
140,WTP,CIP.2596004,Rehabilitation of Quebradillas WTP and RWI,2022-Q2,2024-Q2,$28.84,4,Near-Term
10,D&T-WL,CIP.1019001,"Rehabilitation of D&T-WL Bo. Sonadora, Renacer...",2023-Q1,2023-Q4,$0.65,1,Near-Term
